In [1]:
import sys
sys.path.append("../")

In [4]:
float('1 to 3'.split('to')[1])

3.0

In [2]:
from core.helpers import degree2file
import json

transcript = json.load(open('../aiadvisor/transcripts/1.json'))
transcript

/Users/tarosh/projects/better-section-tally/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'name': 'Tarosh Gurramkonda',
 'birth_date': '10/09/2002',
 'major': 'Computer Science',
 'concentration': 'Artiﬁcial Intelligence',
 'attempted_credits': 118.0,
 'passed_credits': 108.0,
 'earned_credits': 108.0,
 'gpa_credits': 84.0,
 'quality_points': 302.1,
 'gpa': 3.596,
 'transfer': [{'subject': 'ENGL',
   'course_number': '02075',
   'title': 'GE LITERATURE ELECT',
   'grade': 'TR',
   'credits': '3.000'},
  {'subject': 'FREN',
   'course_number': '02101',
   'title': 'ELEMENTARY FRENCH I',
   'grade': 'TR',
   'credits': '3.000'},
  {'subject': 'FREN',
   'course_number': '02102',
   'title': 'ELEMENTARY FRENCH II',
   'grade': 'TR',
   'credits': '3.000'},
  {'subject': 'FREN',
   'course_number': '02201',
   'title': 'INTERMED FRENCH I',
   'grade': 'TR',
   'credits': '3.000'},
  {'subject': 'INTR',
   'course_number': '99013',
   'title': 'DUPLICATED TRANSFER CREDIT',
   'grade': 'TR',
   'credits': '0.000'},
  {'subject': 'INTR',
   'course_number': '99013',
   'title': '

In [3]:
# https://sites.rowan.edu/registrar/services-resources/grading-system-gpa.html
QUALITY_PTS = {
    'A': 4,
    'A-': 3.7,
    'B+': 3.3,
    'B': 3,
    'B-': 2.7,
    'C+': 2.3,
    'C': 2,
    'C-': 1.7,
    'D+': 1.3,
    'D': 1,
    'D-': 0.7,
    'F': 0
}

In [4]:
def has_passed(qual_pts: float, credits: float):
    # if credits is 0, then we know this class is passed
    if credits < 0.0001:
        return True
    
    # minimum C-
    return qual_pts / credits >= QUALITY_PTS['C-']

In [5]:
completed = {}
if 'transfer' in transcript:
    for course in transcript['transfer']:
        if float(course['credits']) > 0:
            completed[f"{course['subject']} {course['course_number']}"] = float(course['credits'])

for term in transcript['completed']:
    for course in term['courses']:
        if has_passed(float(course['quality_points']), float(course['credits'])):
            completed[f"{course['subject']} {course['course_number']}"] = float(course['credits'])

completed

{'ENGL 02075': 3.0,
 'FREN 02101': 3.0,
 'FREN 02102': 3.0,
 'FREN 02201': 3.0,
 'MATH 01131': 4.0,
 'PHYS 00210': 4.0,
 'PHYS 00211': 4.0,
 'COMP 01111': 3.0,
 'CS 00100': 1.0,
 'CS 04113': 4.0,
 'MATH 01130': 4.0,
 'MATH 03150': 3.0,
 'COMP 01112': 3.0,
 'CS 01395': 3.0,
 'CS 04114': 3.0,
 'CS 06205': 3.0,
 'CS 07210': 3.0,
 'MUSG 06109': 3.0,
 'CS 01205': 3.0,
 'INTR 01265': 3.0,
 'MATH 01210': 3.0,
 'PHYS 00220': 4.0,
 'CMS 04205': 3.0,
 'CS 03351': 3.0,
 'STAT 02290': 3.0,
 'WA 01302': 3.0,
 'CS 04222': 4.0,
 'CS 02480': 3.0,
 'CS 06395': 3.0,
 'CS 07340': 3.0,
 'CS 07459': 3.0,
 'CS 04315': 3.0,
 'CS 04321': 4.0,
 'CS 06420': 3.0}

In [10]:
degree = json.load(open(f'../core/vault/degrees/{degree2file('Bachelor of Science', 'Computer Science')}'))
print('name:', degree['name'])
print('heads:', list(degree['content'].keys()))

name: Bachelor of Science in Computer Science
heads: ['Bachelor of Science in Computer Science', 'Rowan Core: 9 s.h.', 'Rowan Experience', 'Program Requirements', 'Non-Program Courses: 18 s.h.', 'Foundational Courses: 36-37 s.h.', 'Upper-Level and Capstone Courses: 16 s.h.', 'Computer Science Restricted Electives: 12 s.h.', 'Free Electives: 28 s.h.', 'Total Required Credits for the Program: 120 s.h.']


In [7]:
import re

courses_left = {}

for head in degree['content'].keys():
    # find "11 s.h." or "32-33 s.h." in the section headings
    credits_needed = re.search(r'(\d{1,3})\s?-?\s?((\d{1,3}))?\s+s.h.', head)
    print(f"head: {head}")
    has_requirements = 'requirements' in degree['content'][head]

    # if required credits is in section heading, and it contains requirements (a.k.a courses) under heading
    if credits_needed and has_requirements:
        requirements = degree['content'][head]['requirements']
        print(f"credits_needed: {credits_needed.group(1)}")
        
        # extract the required credits to complete this section
        _frst = credits_needed.group(1)
        _scnd = credits_needed.group(2)

        # if we find 11 s.h., this would be (11, 11) credits
        # if we find 11-12 s.h., this would be (11, 12) credits
        _frst = float(_frst)
        _scnd = float(_scnd) if _scnd else _frst

        # initalize to accumulate unfinished courses in the section
        courses_left[head] = {
            'total_credits': [_frst, _scnd],
            'completed': False,
            'completed_credits': 0,
            'not_completed': []
        }

        for block in requirements:
            req_crses = [f'{crse['subject']} {crse['course_number']}' for crse in block['courses']]
            completed_crses = set(completed) & set(req_crses)     

            # if there is any completed course in the or block, then the block is satisfied 
            if block['type'] == 'or':
                courses_left[head]['not_completed'] += [] if len(completed_crses) > 0 else req_crses 
            else:
                courses_left[head]['not_completed'] += list(set(req_crses) - completed_crses)

            courses_left[head]['completed_credits'] += sum([completed[crse] for crse in completed_crses])

        # ensure that we have met the credits needed to complete this section
        if courses_left[head]['completed_credits'] >= _frst or courses_left[head]['completed_credits'] >= _scnd:
            courses_left[head]['completed'] = True

head: Bachelor of Science in Computer Science
head: Rowan Core: 9 s.h.
credits_needed: 9
head: Rowan Experience
head: Program Requirements
head: Non-Program Courses: 18 s.h.
credits_needed: 18
head: Foundational Courses: 36-37 s.h.
credits_needed: 36
head: Upper-Level and Capstone Courses: 16 s.h.
credits_needed: 16
head: Computer Science Restricted Electives: 12 s.h.
credits_needed: 12
head: Free Electives: 28 s.h.
head: Total Required Credits for the Program: 120 s.h.
credits_needed: 120


In [8]:
courses_left

{'Rowan Core: 9 s.h.': {'total_credits': [9.0, 9.0],
  'completed': False,
  'completed_credits': 0,
  'not_completed': []},
 'Non-Program Courses: 18 s.h.': {'total_credits': [18.0, 18.0],
  'completed': False,
  'completed_credits': 14.0,
  'not_completed': []},
 'Foundational Courses: 36-37 s.h.': {'total_credits': [36.0, 37.0],
  'completed': False,
  'completed_credits': 30.0,
  'not_completed': ['CS 04215', 'CS 06210']},
 'Upper-Level and Capstone Courses: 16 s.h.': {'total_credits': [16.0, 16.0],
  'completed': False,
  'completed_credits': 13.0,
  'not_completed': ['CS 04400']},
 'Computer Science Restricted Electives: 12 s.h.': {'total_credits': [12.0,
   12.0],
  'completed': True,
  'completed_credits': 15.0,
  'not_completed': ['CS 08360',
   'CS 01303',
   'CS 99300',
   'CS 09410',
   'CS 06447',
   'CS 07460',
   'CS 06390',
   'CS 09415',
   'CS 02370',
   'CS 04444',
   'CS 99490',
   'CS 08380',
   'DS 02395',
   'CS 04471',
   'CS 09427',
   'CS 07310',
   'CS 04376'